# Desafio: Consumo de Dados para Previsão do Tempo das Cidades do Vale do Paraíba.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Neste desafio, você desenvolverá um notebook que será responsável por extrair dados de previsão do tempo das cidades do Vale do Paraíba, região onde se localiza a Dataside. Para consultar todas as cidades dessa região, utilizaremos a API do IBGE. No caso, basta realizar uma requisição HTTP com o método GET, utilizando a URL abaixo:

```
https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios
```

Com esses dados, gerar um data frame e a partir dele uma temp view. Ex: "cities"

Utilizando os nomes das cidades, deverão ser consultados os dados de previsão de tempo para cada cidade. Para realizar essa consulta, poderá ser utilizada qualquer uma das APIs informadas no link abaixo.

[Public APIs - Wather](https://github.com/public-apis/public-apis#weather)

Obs.: Para algumas, pode ser necessário cadastrar-se para acessar sua API Key. Mas nenhuma delas deve precisar cadastrar cartão de crédito ou adicionar qualquer valor monetário para utilizar. Caso alguma solicite, basta optar por outra.

Com os dados consultados, gerar um data frame e partir dele outra temp view. Ex: "forecasts"

Com as temp views geradas, utilizar Spark SQL para criar queries e gerar data frames das seguintes tabelas:

- Tabela 1: dados de previsão do tempo para os próximos cinco dias, para cada data e cidade consultadas. As colunas dessa tabela serão:
    - Cidade
    - CodigoDaCidade
    - Data
    - Regiao
    - Pais
    - Latitude
    - Longigute
    - TemperaturaMaxima
    - TemperaturaMinima
    - TemperaturaMedia
    - VaiChover
    - ChanceDeChuva
    - CondicaoDoTempo
    - NascerDoSol
    - PorDoSol
    - VelocidadeMaximaDoVento
    
    Obs.: Os valores da coluna "VaiChover" deverá ser "Sim" ou "Não". E a coluna "CodigoDaCidade" é o ID retornado junto com os nomes da cidades na API do IBGE.
    Obs.: Dependendo da API utilizada, algumas colunas podem não existir e ficarão em branco. Você deve optar por uma API que traga o maior número de informações possível.

- Tabela 2: quantidade de dias com chuva e sem chuva para os dias consultados, para cada data consultada. Colunas:
    - Cidade
    - QtdDiasVaiChover
    - QtdDiasNaoVaiChover
    - TotalDiasMapeados

Essas tabelas deverão ser exportadas em formado CSV e entregue no final do desafio.

## To Do

[ ] - Consultar municípios do Vale do Paraíba, gerar um data frame e criar uma temp view com esses dados.
[ ] - Consultar dados do tempo para cada município, gerar um data frame e criar uma outra temp view.
[ ] - Utilizar Spark SQL para gerar os data frames das Tabelas 1 e 2.
[ ] - Exportar os data frames para CSV.

## Atenção

- Existe um limite de requisições de 10000 requests por conta cadastrada na m3o.
- Essa API pode retornar cidades de outras regiões que possuem nome semelhante a alguma cidade do Vale do Paraiba. Pode mantê-las ou filtrar para gerar as tabelas apenas com dados de Regiao = Sao Paulo. Fica a seu critério.

## Entregando o desafio

Concluindo todos os passos informados em To Do, basta salvar o arquivo .ipynb do notebook e enviar para a Dataside juntamente com os CSVs das duas tabelas.


In [1]:
import findspark
findspark.init()

import pandas as pd
import requests
import json
import unidecode
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[2]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

In [8]:
# Buscar cidades do Vale do Paraíba
# TODO

request = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios")

todos = json.loads(request.content)

# Criar data frame com as cidades
# TODO

df_cities = pd.DataFrame.from_dict(todos)

listCidades = []

#Colocando as cidades em uma lista
for cidade in df_cities["nome"]:
    listCidades.append(cidade)
print(listCidades)

# Criar view com as cidades
# TODO

cities_csv = df_cities.to_csv("cities.csv")

data =  spark.read.format("csv").option("header","true").option("inferSchema","true").load("cities.csv")

data.createOrReplaceTempView("cities")

query_all_info = "SELECT * from cities"

query_only_cities = "SELECT nome from cities"

all_info_cities = spark.sql(query_all_info)

only_cities = spark.sql(query_only_cities)

# #Exibe toda a tabela
# all_info_cities.show()

# #Exibe apenas id e nome da cidade
# only_cities.show()



['Aparecida', 'Arapeí', 'Areias', 'Bananal', 'Caçapava', 'Cachoeira Paulista', 'Campos do Jordão', 'Canas', 'Caraguatatuba', 'Cruzeiro', 'Cunha', 'Guaratinguetá', 'Igaratá', 'Ilhabela', 'Jacareí', 'Jambeiro', 'Lagoinha', 'Lavrinhas', 'Lorena', 'Monteiro Lobato', 'Natividade da Serra', 'Paraibuna', 'Pindamonhangaba', 'Piquete', 'Potim', 'Queluz', 'Redenção da Serra', 'Roseira', 'Santa Branca', 'Santo Antônio do Pinhal', 'São Bento do Sapucaí', 'São José do Barreiro', 'São José dos Campos', 'São Luiz do Paraitinga', 'São Sebastião', 'Silveiras', 'Taubaté', 'Tremembé', 'Ubatuba']


In [57]:
# Buscar previsão do tempo para as cidades
# TODO

def getPrevisao():
    
    API_KEY = "6e0fb0f7945da3a636577cf29162aa2e"
    
    listaPrevisao = []
    
    for i in listCidades:

            city = i

            lang = "pt_br"

            cnt = 40

            api_url_base = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&cnt={cnt}&appid={API_KEY}&lang=pt_br"

            request_api = requests.get(api_url_base)

            request_api_dic = request_api.json()


            # Armazenando os dados do json em variáveis
            Cidade = city
            CodigoDaCidade = "null"

            Data = request_api_dic["list"][0]["dt_txt"]
            Data2 = request_api_dic["list"][8]["dt_txt"]
            Data3 = request_api_dic["list"][16]["dt_txt"]
            Data4 = request_api_dic["list"][24]["dt_txt"]
            Data5 = request_api_dic["list"][32]["dt_txt"]

            Regiao = "null" 
            Pais = request_api_dic["city"]["country"]
            Latitude = request_api_dic["city"]["coord"]["lat"]
            Longigute = request_api_dic["city"]["coord"]["lon"]

            TemperaturaMaxima = request_api_dic["list"][0]["main"]["temp_max"] - 273.15
            TemperaturaMaxima2 = request_api_dic["list"][8]["main"]["temp_max"] - 273.15
            TemperaturaMaxima3 = request_api_dic["list"][16]["main"]["temp_max"] - 273.15
            TemperaturaMaxima4 = request_api_dic["list"][24]["main"]["temp_max"] - 273.15
            TemperaturaMaxima5 = request_api_dic["list"][32]["main"]["temp_max"] - 273.15

            TemperaturaMinima = request_api_dic["list"][0]["main"]["temp_min"] - 273.15
            TemperaturaMinima2 = request_api_dic["list"][8]["main"]["temp_min"] - 273.15
            TemperaturaMinima3 = request_api_dic["list"][16]["main"]["temp_min"] - 273.15
            TemperaturaMinima4 = request_api_dic["list"][24]["main"]["temp_min"] - 273.15
            TemperaturaMinima5 = request_api_dic["list"][32]["main"]["temp_min"] - 273.15

            TemperaturaMedia = request_api_dic["list"][0]["main"]["temp"] - 273.15
            TemperaturaMedia2 = request_api_dic["list"][8]["main"]["temp"] - 273.15
            TemperaturaMedia3 = request_api_dic["list"][16]["main"]["temp"] - 273.15
            TemperaturaMedia4 = request_api_dic["list"][24]["main"]["temp"] - 273.15
            TemperaturaMedia5 = request_api_dic["list"][32]["main"]["temp"] - 273.15

            oldVaiChover = request_api_dic["list"][0]["pop"] 
            vaiChover = "não" if oldVaiChover == 0 else "sim"

            oldVaiChover2 = request_api_dic["list"][8]["pop"] 
            vaiChover2= "não" if oldVaiChover2 == 0 else "sim"

            oldVaiChover3 = request_api_dic["list"][16]["pop"] 
            vaiChover3= "não" if oldVaiChover3 == 0 else "sim"

            oldVaiChover4 = request_api_dic["list"][24]["pop"] 
            vaiChover4= "não" if oldVaiChover4 == 0 else "sim"

            oldVaiChover5 = request_api_dic["list"][32]["pop"] 
            vaiChover5= "não" if oldVaiChover5 == 0 else "sim"

            ChanceDeChuva = request_api_dic["list"][0]["pop"]
            ChanceDeChuva2 = request_api_dic["list"][8]["pop"]
            ChanceDeChuva3 = request_api_dic["list"][16]["pop"]
            ChanceDeChuva4 = request_api_dic["list"][24]["pop"]
            ChanceDeChuva5 = request_api_dic["list"][32]["pop"]

            CondicaoDoTempo = request_api_dic["list"][0]["weather"][0]["description"]
            CondicaoDoTempo2 = request_api_dic["list"][0]["weather"][0]["description"]
            CondicaoDoTempo3 = request_api_dic["list"][0]["weather"][0]["description"]
            CondicaoDoTempo4 = request_api_dic["list"][0]["weather"][0]["description"]
            CondicaoDoTempo5 = request_api_dic["list"][0]["weather"][0]["description"]

            NascerDoSol = "null"
            PorDoSol = "null"

            VelocidadeMaximaDoVento = request_api_dic["list"][0]["wind"]["speed"] * 3.6
            VelocidadeMaximaDoVento2 = request_api_dic["list"][8]["wind"]["speed"]  * 3.6
            VelocidadeMaximaDoVento3 = request_api_dic["list"][16]["wind"]["speed"] * 3.6
            VelocidadeMaximaDoVento4 = request_api_dic["list"][24]["wind"]["speed"] * 3.6
            VelocidadeMaximaDoVento5 = request_api_dic["list"][32]["wind"]["speed"] * 3.6
   
            # Criando dicionario com a previsao
            previsao_dic = {"Data":[Data,Data2,Data3,Data4,Data5], 
                            "Cidade":[Cidade,Cidade,Cidade,Cidade,Cidade],
                            "CodigoDaCidade":[CodigoDaCidade,CodigoDaCidade,CodigoDaCidade,CodigoDaCidade,CodigoDaCidade],
                            "Regiao":[Regiao,Regiao,Regiao,Regiao,Regiao],
                            "Pais":[Pais,Pais,Pais,Pais,Pais],
                            "Latitude":[Latitude,Latitude,Latitude,Latitude,Latitude],
                            "Longigute":[Longigute,Longigute,Longigute,Longigute,Longigute],
                            "TemperaturaMaxima":[TemperaturaMaxima,TemperaturaMaxima2,TemperaturaMaxima3,TemperaturaMaxima4,TemperaturaMaxima5],
                            "TemperaturaMinima":[TemperaturaMinima,TemperaturaMinima2,TemperaturaMinima3,TemperaturaMinima4,TemperaturaMinima5],
                            "TemperaturaMedia":[TemperaturaMedia,TemperaturaMedia2,TemperaturaMedia3,TemperaturaMedia4,TemperaturaMedia5],
                            "VaiChover":[vaiChover,vaiChover2,vaiChover3,vaiChover4,vaiChover5],
                            "ChanceDeChuva":[ChanceDeChuva,ChanceDeChuva2,ChanceDeChuva3,ChanceDeChuva4,ChanceDeChuva5],
                            "CondicaoDoTempo":[CondicaoDoTempo,CondicaoDoTempo2,CondicaoDoTempo3,CondicaoDoTempo4,CondicaoDoTempo5],
                            "NascerDoSol":[NascerDoSol,NascerDoSol,NascerDoSol,NascerDoSol,NascerDoSol],
                            "PorDoSol":[PorDoSol,PorDoSol,PorDoSol,PorDoSol,PorDoSol],
                            "VelocidadeMaximaDoVento":[VelocidadeMaximaDoVento,VelocidadeMaximaDoVento2,VelocidadeMaximaDoVento3,VelocidadeMaximaDoVento4,VelocidadeMaximaDoVento5]}              
          
            listaPrevisao.append(previsao_dic)

    return listaPrevisao

# Criar data frame com as previsões
 
final = getPrevisao()

### lista para json
jsonStr = json.dumps(final)

### json para dic
dicJson = json.loads(jsonStr)

## dic para DF
df_previsao = pd.DataFrame.from_dict(dicJson)

#dataframe para csv
previsao_csv = df_previsao.to_csv("previsao.csv") 

#Temp View da previsão
dados =  spark.read.format("csv").option("header","true").option("inferSchema","true").load("previsao.csv")

dados.createOrReplaceTempView("previsao")

query_all_previsao = "SELECT * from tabela1"

all_info_previsao = spark.sql(query_all_previsao)

#exibir a temp view
# all_info_previsao.show()



In [58]:
# Criar DF da Tabela 1
df_tabela1 = pd.DataFrame.from_dict(dicJson)
df_tabela1


In [ ]:
# Criar DF da Tabela 2
# TODO

In [ ]:
# Exportar CSVs
tabela1_csv = df_tabela1.to_csv("tabela1.csv") 